In [1]:
import pandas as pd
import numpy as np

In [2]:
churn_data=pd.read_csv('telecom_churn_data.csv')

In [3]:
churn_data.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


In [4]:
churn_data.shape

(99999, 226)

In [5]:
cols=churn_data.columns
cols

Index(['mobile_number', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou',
       'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7',
       'last_date_of_month_8', 'last_date_of_month_9', 'arpu_6',
       ...
       'sachet_3g_9', 'fb_user_6', 'fb_user_7', 'fb_user_8', 'fb_user_9',
       'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g', 'sep_vbc_3g'],
      dtype='object', length=226)

In [6]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


Columns with more than 40% null values.

In [7]:
#Columns with more than 40% null values.
null_series=round(100*(churn_data.isnull().sum(axis=0)/len(churn_data)),2).where(lambda x:x!=0).dropna()
null_series.sort_values(ascending=False).where(lambda x: x>40).dropna()

count_rech_3g_6             74.85
fb_user_6                   74.85
date_of_last_rech_data_6    74.85
count_rech_2g_6             74.85
night_pck_user_6            74.85
av_rech_amt_data_6          74.85
total_rech_data_6           74.85
arpu_2g_6                   74.85
max_rech_data_6             74.85
arpu_3g_6                   74.85
arpu_3g_7                   74.43
count_rech_3g_7             74.43
count_rech_2g_7             74.43
av_rech_amt_data_7          74.43
max_rech_data_7             74.43
arpu_2g_7                   74.43
night_pck_user_7            74.43
date_of_last_rech_data_7    74.43
fb_user_7                   74.43
total_rech_data_7           74.43
total_rech_data_9           74.08
count_rech_2g_9             74.08
max_rech_data_9             74.08
date_of_last_rech_data_9    74.08
fb_user_9                   74.08
count_rech_3g_9             74.08
night_pck_user_9            74.08
av_rech_amt_data_9          74.08
arpu_2g_9                   74.08
arpu_3g_9     

Columns with only 1 unique value.

In [8]:
# Columns with only 1 unique value.
nunique_1=pd.DataFrame(churn_data.nunique().where(lambda x:x==1).dropna())
nunique_1.index

Index(['circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou',
       'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8',
       'last_date_of_month_9', 'std_og_t2c_mou_6', 'std_og_t2c_mou_7',
       'std_og_t2c_mou_8', 'std_og_t2c_mou_9', 'std_ic_t2o_mou_6',
       'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8', 'std_ic_t2o_mou_9'],
      dtype='object')

# Data Preparation

In [9]:
#Calculating the average ARPU for month 6 and 7.
churn_data['avg']=(churn_data['arpu_6']+churn_data['arpu_7'])/2

In [10]:
# Calculating the 70th Percentile of average recharge amount in the first two months
avg=churn_data['avg'].quantile(0.70)
print('70th percentile of the average recharge amount in the first two months = ',avg)

70th percentile of the average recharge amount in the first two months =  317.30929999999984


In [11]:
# Number of High Value Customer(who recharge more than 70 percentile of average)
churn_data['mobile_number'].where(churn_data['avg']>avg).dropna().shape

(30000,)

In [40]:
# Cretaing a new dataset for High Value Customer only
churn_data_new=churn_data[churn_data['mobile_number'].where(churn_data['avg']>avg).isnull() == False]
churn_data_new.shape

(30000, 227)

In [41]:
# Replacing the Null values with 0.
churn_data_new.fillna(0,inplace=True)

C:\Users\USER.000\Anaconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [42]:
# Checking if any column has Null values.
churn_data_new.isnull().sum().where(lambda x: x>0).dropna()

Series([], dtype: float64)

In [43]:
filter1=churn_data_new['total_ic_mou_9']==0 
filter2=churn_data_new['total_og_mou_9']==0 
filter3=churn_data_new['vol_2g_mb_9']==0 
filter4=churn_data_new['vol_3g_mb_9']==0

In [44]:
churn_data_new.loc[filter1 & filter2 & filter3 & filter4,'churn']=1

C:\Users\USER.000\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\USER.000\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [45]:
churn_data_new['churn'].fillna(0,inplace=True)

C:\Users\USER.000\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [49]:
cols=[col for col in churn_data_new.columns if "_9" in col]
print("Number of columns with _9 as part of it = ",len(cols))

Number of columns with _9 as part of it =  0


In [47]:
churn_data_new.drop(cols,axis=1,inplace=True)

In [48]:
churn_data_new.shape

(30000, 174)

In [53]:
churn_data_new['churn'].value_counts()

0.0    27348
1.0     2652
Name: churn, dtype: int64